In [1]:
import tensorflow as tf
from tensorflow.keras.layers import InputLayer, Rescaling
from tensorflow.keras.models import Sequential
import os

# === CONFIGURE PATHS ===
KERAS_MODEL_PATH = r"D:\iot project\final_finetuned_model.keras"
CALIB_DIR        = r"D:\iot project\train"
OUTPUT_TFLITE    = "animal_int8.tflite"
IMG_H, IMG_W     = 96, 96

# === LOAD TRAINED MODEL ===
model = tf.keras.models.load_model(KERAS_MODEL_PATH)

# === WRAP MODEL TO ACCEPT UINT8 INPUT ===
wrapped_model = Sequential([
    InputLayer(input_shape=(IMG_H, IMG_W, 3), dtype=tf.uint8),
    Rescaling(1./255),
    model
])

# === COLLECT CALIBRATION DATA ===
rep_paths = []
for root, _, files in os.walk(CALIB_DIR):
    for fname in files:
        if fname.lower().endswith((".jpg", ".jpeg", ".png")):
            rep_paths.append(os.path.join(root, fname))

print("Calibration images found:", len(rep_paths))
if len(rep_paths) == 0:
    raise RuntimeError("No images found for calibration!")

def representative_dataset_generator():
    for img_path in rep_paths[:100]:  # Limit to 100 images for speed
        img = tf.io.read_file(img_path)
        img = tf.image.decode_image(img, channels=3)
        img = tf.image.resize(img, [IMG_H, IMG_W])
        img = tf.cast(img, tf.uint8)
        img = tf.expand_dims(img, 0)
        yield [img]

# === CONVERT TO TFLITE INT8 ===
converter = tf.lite.TFLiteConverter.from_keras_model(wrapped_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
converter.representative_dataset = representative_dataset_generator

tflite_quant = converter.convert()

# === SAVE TFLITE MODEL ===
with open(OUTPUT_TFLITE, "wb") as f:
    f.write(tflite_quant)

size_kb = os.path.getsize(OUTPUT_TFLITE) / 1024
print(f"✅ Fully quantized model saved to '{OUTPUT_TFLITE}' ({size_kb:.1f} KB)")


ValueError: File not found: filepath=D:\iot project\final_finetuned_model.keras. Please ensure the file is an accessible `.keras` zip file.